# Deploy the PowerSkill to Azure Search

* The first step is to upload the data files in the data folder to a container in Azure blob storage and
get the connection values to create the ACS data source.
* You will need various properties, such as your [ACS API Key](https://docs.microsoft.com/en-us/azure/search/search-security-api-keys) and the URL for your ACS instance to be configured in the `.env` file in the `notebooks` folder

In [ ]:
from dotenv import load_dotenv
import os

## NOTE: This notebook loads parameters from the .env file in the notebooks folder.
##       This are normally generated by the deployment script - see README.md

load_dotenv(dotenv_path='.env')

WEB_APP_KEY = os.getenv("WEB_APP_KEY")   # Set the KEY value you deployed your Web App with [YourSecretKeyCanBeAnything]
WEB_APP_URL = os.getenv("WEB_APP_URL")   # This is the deployed web app format: URL https://[appname].azurewebsites.net

SEARCH_API_KEY = os.getenv("SEARCH_API_KEY")  # Your ACS API Key
ACS_URL        = os.getenv("ACS_URL")         # Your ACS URL format https://[your ACS instance].search.windows.net
DATA_SOURCE    = os.getenv("DATA_SOURCE")     # The name for your data source, e.g. clustering-datasource

SKILLSET_NAME = os.getenv("SKILLSET_NAME")  # The name of your skillset, e.g. testclusteringskillset
COGSVC_KEY    = os.getenv("COGSVC_KEY")     # This is your Cognitive Services key that resides in the same region as ACS (used to compare custom vision captions and object detection)


INDEX_NAME = os.getenv("INDEX_NAME")  # The name for the index, e.g.  clustering-index

STORAGE_CONNECTION_STRING = os.getenv("STORAGE_CONNECTION_STRING") # The connection string for the storage account with the images
STORAGE_CONTAINER_NAME = os.getenv("STORAGE_CONTAINER_NAME") # The name of the container with the images



# Create the data source

In [ ]:
# Let's create a data source

json_text = {
    "name" : DATA_SOURCE,
    "type" : "azureblob",
    "credentials" : { "connectionString" : ""},    #  This is your azure blob connection string
    "container" : { "name" : "books" }  # The name of the container where the data files are
}

headers = {
    "api-key": SEARCH_API_KEY,     
    "Content-Type": "application/json",

}

try:
    url = f"{ACS_URL}/datasources?api-version=2020-06-30"
    resp = post(url=url, json=json_text, headers=headers)
    
    result_response = resp.json()
    if resp.status_code == 403:
        print("Authorisation Failed: Check that your API KEY value is correct")
        
        
    if resp.status_code == 201:
        print("Success creating data source")
        
except Exception as e:
    print('Exception creating data source', e)

# Now we create the index

In [ ]:

json_text = {
      "name" : INDEX_NAME,
      "fields": [
        { "name": "id", "type": "Edm.String", "key": "true", "searchable": "false" },
        { "name": "url", "type": "Edm.String", "searchable": "true" },
        { "name": "file_name", "type": "Edm.String", "searchable": "false" },
        { "name": "size", "type": "Edm.Int64", "searchable": "false" },
        { "name": "last_modified", "type": "Edm.DateTimeOffset", "searchable": "false" },
        { "name": "content", "type": "Edm.String", "searchable": "true", "filterable": "false", "sortable": "false", "facetable": "false" },
        { "name": "label", "type": "Collection(Edm.String)", "searchable": "true", "filterable": "false", "sortable": "false", "facetable": "false" }
      ]
}

try:
    url = f"{ACS_URL}/indexes?api-version=2020-06-30"
    resp = post(url=url, json=json_text, headers=headers)
    
    result_response = resp.json()
    if resp.status_code == 403:
        print("Authorisation Failed: Check that your API KEY value is correct")
        
        
    if resp.status_code == 201:
        print("Success creating index")
        
except Exception as e:
    print('Exception creating index', e)

# Now we create the skill set

In [ ]:
# Note we are passing in the secret header key and the inference API URL to the skillset

json_text = {
    "description": "Cluster images for labelling",
    "skills": [
        {
            "@odata.type": "#Microsoft.Skills.Custom.WebApiSkill",
            "description": "A custom skill that can cluster and auto label images",
            "uri": f"{WEB_APP_URL}/api/extraction",
            "timeout": "PT160S",
            "batchSize": 1,
            "context": "/document/normalized_images/*",
            "httpHeaders": {
                "Ocp-Apim-Subscription-Key": SEARCH_API_KEY
            },
            "httpMethod": "POST",
            "inputs": [
            {
                "name": "images",
                "source": "/document/normalized_images/*"
            }
            ],
            "outputs": [
                {
                    "name": "label"
                }
            ]
        }
    ],
    "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": "cogsvc",
        "key": COGSVC_KEY
    }
}


try:
    url = f"{ACS_URL}/skillsets/{SKILLSET_NAME}?api-version=2020-06-30"
    resp = put(url=url, json=json_text, headers=headers)
    
    result_response = resp.json()
    if resp.status_code == 403:
        print("Authorisation Failed: Check that your API KEY value is correct")

    if resp.status_code == 201:
        print("Success creating skillset")
        
except Exception as e:
    print('Exception creating skillset', e)


# Now we create the indexer

In [ ]:
json_text = {
    "name": "clustering-indexerpr",
    "dataSourceName": DATA_SOURCE,
    "targetIndexName": INDEX_NAME,
    "skillsetName": SKILLSET_NAME,
    "parameters": {
    "configuration": {
      "allowSkillsetToReadFileData": "true",
      "imageAction": "generateNormalizedImages"
    }
    },
    "outputFieldMappings": [
        {
            "sourceFieldName": "/document/normalized_images/*/label",
            "targetFieldName": "label"
        }
    ]

}


try:
    url = f"{ACS_URL}/indexers?api-version=2020-06-30"
    resp = post(url=url, json=json_text, headers=headers)
    
    result_response = resp.json()
    if resp.status_code == 403:
        print("Authorisation Failed: Check that your API KEY value is correct")

    if resp.status_code == 201:
        print("Success creating indexer")
        
except Exception as e:
    print('Exception creating indexer', e)